In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
#loading the CIFAR10 dataset

(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)

Dataset cifar10 downloaded and prepared to C:\Users\91700\tensorflow_datasets\cifar10\3.0.2. Subsequent calls will reuse this data.


In [3]:
def normalize_img(image,label):
    return tf.cast(image,tf.float32)/255.0, label

In [4]:
AUTOTUNE= tf.data.experimental.AUTOTUNE
BATCH_SIZE= 32

In [5]:
def augment(image, label):
    new_height= new_width = 32
    image= tf.image.resize(image,(new_height,new_width))
    
    # only effect images after a threshold
    if tf.random.uniform((), minval=0,maxval=1)<0.1:
        image=tf.tile(tf.image.rgb_to_grayscale(image),[1,1,3])
        
    image=tf.image.random_brightness(image, max_delta=0.1)
    image=tf.image.random_contrast(image, lower=0.1, upper=0.2)
    
    # to flip and stuff
    image=tf.image.random_flip_left_right(image)
    
    return image, label

In [6]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(height=32, width=32,),
        layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
        layers.experimental.preprocessing.RandomContrast(factor=0.1,),
    ]
)

model = keras.Sequential(
    [
        keras.Input((32, 32, 3)),
        data_augmentation,
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

In [7]:
model = keras.Sequential(
    [
        keras.Input((32, 32, 3)),
        data_augmentation,
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

In [8]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test)

Epoch 1/5
1563/1563 - 16s - loss: 1.6470 - accuracy: 0.4142
Epoch 2/5
1563/1563 - 7s - loss: 1.3510 - accuracy: 0.5239
Epoch 3/5
1563/1563 - 8s - loss: 1.2533 - accuracy: 0.5601
Epoch 4/5
1563/1563 - 8s - loss: 1.1853 - accuracy: 0.5835
Epoch 5/5
1563/1563 - 8s - loss: 1.1305 - accuracy: 0.6040
1563/1563 [==============================] - 7s 4ms/step - loss: 1.0844 - accuracy: 0.6191


[1.0844171047210693, 0.6190999746322632]